# import libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

# Load Data

In [3]:
df = pd.read_csv(r"../data/raw/Hotel Reservations.csv")
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


# Data Overview

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  str    
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  str    
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  str    
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date              

In [5]:
# check summary numerical statistics
df.describe(include="number").round(2)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00
mean,1.84,0.11,0.81,2.20,0.03,85.23,2017.82,7.42,15.60,0.03,0.02,0.15,103.42,0.62
std,0.52,0.40,0.87,1.41,0.17,85.93,0.38,3.07,8.74,0.16,0.37,1.75,35.09,0.79
min,0.00,0.00,0.00,0.00,0.00,0.00,2017.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,2.00,0.00,0.00,1.00,0.00,17.00,2018.00,5.00,8.00,0.00,0.00,0.00,80.30,0.00
50%,2.00,0.00,1.00,2.00,0.00,57.00,2018.00,8.00,16.00,0.00,0.00,0.00,99.45,0.00
75%,2.00,0.00,2.00,3.00,0.00,126.00,2018.00,10.00,23.00,0.00,0.00,0.00,120.00,1.00
max,4.00,10.00,7.00,17.00,1.00,443.00,2018.00,12.00,31.00,1.00,13.00,58.00,540.00,5.00


In [6]:
# check summary categorical statistics
df.describe(include="O").round(2)

C:\Users\work\AppData\Local\Temp\ipykernel_34012\1868845830.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  df.describe(include="O").round(2)


,Booking_ID,type_of_meal_plan,room_type_reserved,market_segment_type,booking_status
count,36275,36275,36275,36275,36275
unique,36275,4,7,5,2
top,INN00001,Meal Plan 1,Room_Type 1,Online,Not_Canceled
freq,1,27835,28130,23214,24390


In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='str')

# Feature Engineering

Total guests

In [9]:
df["total_guests"] = df["no_of_adults"] + df["no_of_children"]

Total nights

In [10]:
df["total_nights"] = df["no_of_weekend_nights"] + df["no_of_week_nights"]

In [11]:
df.loc[df['total_nights'] == 0, 'total_nights'] = 1

price per night

In [12]:
df['price_per_night'] = df['avg_price_per_room'] / df['total_nights']

is weekend only

In [13]:
df['is_weekend_only'] = (df['total_nights'] == df['no_of_weekend_nights']).astype(int)

In [14]:
df.to_csv('final_data.csv', index=False)

# Preprocessing

In [15]:
x = df.drop(columns=["booking_status","Booking_ID"])
y = df["booking_status"]

In [16]:
x.columns

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'total_guests',
       'total_nights', 'price_per_night', 'is_weekend_only'],
      dtype='str')

# Pipelines

In [17]:
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, LabelEncoder
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

In [18]:
st_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'arrival_year', 'arrival_month','arrival_date',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
        'no_of_special_requests','total_guests','total_nights']

In [19]:
ready_cols = ["repeated_guest","required_car_parking_space","is_weekend_only"]

In [20]:
ohe_cols = ["type_of_meal_plan",    "room_type_reserved",   "market_segment_type"]

In [21]:
ro_cols = ["avg_price_per_room", "price_per_night", "lead_time"]

In [22]:
# standard scaler
st_pipe = Pipeline(steps=[("st_scaler",StandardScaler())])
st_pipe

,steps,"[('st_scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [23]:
# Robust scaler
ro_pipe = Pipeline(steps=[("ro_scaler",RobustScaler())])
ro_pipe

,steps,"[('ro_scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"with_centering with_centering: bool, default=TrueIf `True`, center the data before scaling.This will cause :meth:`transform` to raise an exception when attemptedon sparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_scaling with_scaling: bool, default=TrueIf `True`, scale the data to interquartile range.",True
,"quantile_range quantile_range: tuple (q_min, q_max), 0.0 < q_min < q_max < 100.0, default=(25.0, 75.0)Quantile range used to calculate `scale_`. By default this is equal tothe IQR, i.e., `q_min` is the first quantile and `q_max` is the thirdquantile... versionadded:: 0.18","(25.0, ...)"
,"copy copy: bool, default=TrueIf `False`, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"unit_variance unit_variance: bool, default=FalseIf `True`, scale data so that normally distributed features have avariance of 1. In general, if the difference between the x-values of`q_max` and `q_min` for a standard normal distribution is greaterthan 1, the dataset will be scaled down. If less than 1, the datasetwill be scaled up... versionadded:: 0.24",False


In [24]:
# ohe pipeline
ohe_pipe = Pipeline(steps=[("ohe_encoder", OneHotEncoder(drop="first",sparse_output=False))])
ohe_pipe

,steps,"[('ohe_encoder', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values within a single feature, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute... versionadded:: 0.20",'auto'
,"drop drop: {'first', 'if_binary'} or an array-like of shape (n_features,), default=NoneSpecifies a methodology to use to drop one of the categories perfeature. This is useful in situations where perfectly collinearfeatures cause problems, such as when feeding the resulting datainto an unregularized linear regression model.However, dropping one category breaks the symmetry of the originalrepresentation and can therefore induce a bias in downstream models,for instance for penalized linear classification or regression models.- None : retain all features (the default).- 'first' : drop the first category in each feature. If only one category is present, the feature will be dropped entirely.- 'if_binary' : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.- array : ``drop[i]`` is the category in feature ``X[:, i]`` that should be dropped.When `max_categories` or `min_frequency` is configured to groupinfrequent categories, the dropping behavior is handled after thegrouping... versionadded:: 0.21 The parameter `drop` was added in 0.21... versionchanged:: 0.23 The option `drop='if_binary'` was added in 0.23... versionchanged:: 1.1 Support for dropping infrequent categories.",'first'
,"sparse_output sparse_output: bool, default=TrueWhen ``True``, it returns a :class:`scipy.sparse.csr_matrix`,i.e. a sparse matrix in ""Compressed Sparse Row"" (CSR) format... versionadded:: 1.2 `sparse` was renamed to `sparse_output`",False
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'ignore', 'infrequent_if_exist', 'warn'}, default='error'Specifies the way unknown categories are handled during :meth:`transform`.- 'error' : Raise an error if an unknown category is present during transform.- 'ignore' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.- 'infrequent_if_exist' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will map to the infrequent category if it exists. The infrequent category will be mapped to the last position in the encoding. During inverse transform, an unknown category will be mapped to the category denoted `'infrequent'` if it exists. If the `'infrequent'` category does not exist, then :meth:`transform` and :meth:`inverse_transform` will handle an unknown category as with `handle_unknown='ignore'`. Infrequent categories exist based on `min_frequency` and `max_categories`. Read more in the :ref:`User Guide `.- 'warn' : When an unknown category is encountered during transform a warning is issued, and the encoding then proceeds as described for `handle_unknown=""infrequent_if_exist""`... versionchanged:: 1.1 `'infrequent_if_exist'` was added to automatically handle unknown categories and infrequent categories... versionadded:: 1.6 The option `""warn""` was added in 1.6.",'error'
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered in

In [25]:
preprocessing_pipeline = ColumnTransformer(transformers=[
    ("st_pipe",st_pipe,st_cols),
    ("ro_pipe",ro_pipe,ro_cols),
    ("ohe_pipe",ohe_pipe,ohe_cols),
    ("ready_pipe", "passthrough", ready_cols)
    ])

preprocessing_pipeline

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('st_pipe', ...), ('ro_pipe', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_n

In [26]:
le = LabelEncoder()

y_encoded = le.fit_transform(y)

# Machine Learning

## select the best model


### without smote

In [30]:
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

scoring = {
    'F1': make_scorer(f1_score),
    'Recall': make_scorer(recall_score),
    'Precision': make_scorer(precision_score),
    'Accuracy': make_scorer(accuracy_score)
}

models = [
    ('Logistic Regression', LogisticRegression(random_state=42, n_jobs=-1)),
    ('Knn', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42, n_jobs=-1)),
    ('Xgboost', XGBClassifier()),
    ('LightGBM', LGBMClassifier())
]


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models:
    pipe = Pipeline([
        ("preprocessing", preprocessing_pipeline),
        ("model", model)        
    ])



    result = cross_validate(pipe, x, y_encoded, cv=cv, scoring=scoring, return_train_score=True, n_jobs=-1, verbose=0)
        
    print(name)
    print('Train F1      :', result['train_F1'].mean().round(2)*100)
    print('Test F1       :', result['test_F1'].mean().round(2)*100)
    print('Train Recall  :', result['train_Recall'].mean().round(2)*100)
    print('Test Recall   :', result['test_Recall'].mean().round(2)*100)
    print('Train Precision:', result['train_Precision'].mean().round(2)*100)
    print('Test Precision :', result['test_Precision'].mean().round(2)*100)
    print('Train Accuracy :', result['train_Accuracy'].mean().round(2)*100)
    print('Test Accuracy  :', result['test_Accuracy'].mean().round(2)*100)
    print('Total Training Time :', result['fit_time'].sum().round(2))
    print('-'*50)



Logistic Regression
Train F1      : 86.0
Test F1       : 86.0
Train Recall  : 89.0
Test Recall   : 89.0
Train Precision: 83.0
Test Precision : 83.0
Train Accuracy : 81.0
Test Accuracy  : 81.0
Total Training Time : 3.36
--------------------------------------------------
Knn
Train F1      : 92.0
Test F1       : 88.0
Train Recall  : 93.0
Test Recall   : 89.0
Train Precision: 91.0
Test Precision : 87.0
Train Accuracy : 89.0
Test Accuracy  : 84.0
Total Training Time : 0.5
--------------------------------------------------
Gaussian Naive Bayes
Train F1      : 24.0
Test F1       : 24.0
Train Recall  : 14.000000000000002
Test Recall   : 14.000000000000002
Train Precision: 89.0
Test Precision : 89.0
Train Accuracy : 41.0
Test Accuracy  : 41.0
Total Training Time : 0.81
--------------------------------------------------
Decision Tree
Train F1      : 100.0
Test F1       : 90.0
Train Recall  : 100.0
Test Recall   : 89.0
Train Precision: 100.0
Test Precision : 90.0
Train Accuracy : 99.0
Test Accura

### with smote

In [31]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models:
    pipeline = Pipeline([
        ('Preprocessing', preprocessing_pipeline),
        ("SMOTE",SMOTE()),
        ('Model', model)
    ])
    
    result = cross_validate(
        pipeline,
        x, y_encoded,
        cv=cv,
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1
    )
    
    print(name)
    print('Train F1      :', result['train_F1'].mean().round(2)*100)
    print('Test F1       :', result['test_F1'].mean().round(2)*100)
    print('Train Recall  :', result['train_Recall'].mean().round(2)*100)
    print('Test Recall   :', result['test_Recall'].mean().round(2)*100)
    print('Train Precision:', result['train_Precision'].mean().round(2)*100)
    print('Test Precision :', result['test_Precision'].mean().round(2)*100)
    print('Train Accuracy :', result['train_Accuracy'].mean().round(2)*100)
    print('Test Accuracy  :', result['test_Accuracy'].mean().round(2)*100)
    print('Total Training Time :', result['fit_time'].sum().round(2))
    print('-'*50)


Logistic Regression
Train F1      : 83.0
Test F1       : 83.0
Train Recall  : 79.0
Test Recall   : 79.0
Train Precision: 87.0
Test Precision : 87.0
Train Accuracy : 78.0
Test Accuracy  : 78.0
Total Training Time : 9.54
--------------------------------------------------
Knn
Train F1      : 91.0
Test F1       : 86.0
Train Recall  : 87.0
Test Recall   : 82.0
Train Precision: 96.0
Test Precision : 90.0
Train Accuracy : 89.0
Test Accuracy  : 82.0
Total Training Time : 5.44
--------------------------------------------------
Gaussian Naive Bayes
Train F1      : 24.0
Test F1       : 24.0
Train Recall  : 14.000000000000002
Test Recall   : 14.000000000000002
Train Precision: 89.0
Test Precision : 89.0
Train Accuracy : 41.0
Test Accuracy  : 41.0
Total Training Time : 5.86
--------------------------------------------------
Decision Tree
Train F1      : 100.0
Test F1       : 90.0
Train Recall  : 99.0
Test Recall   : 89.0
Train Precision: 100.0
Test Precision : 91.0
Train Accuracy : 99.0
Test Accura

Although Random Forest achieved the highest F1-score, XGBoost demonstrated a smaller generalization gap between training and testing performance, indicating better robustness and reduced overfitting. Additionally, XGBoost required less training time, making it a more efficient and scalable choice. Furthermore, applying SMOTE did not lead to noticeable performance improvements across the evaluated models, as tree-based and boosting algorithms already handled class imbalance effectively. Given the marginal performance difference, XGBoost can be considered a strong alternative and, in practical scenarios, a more reliable model.